# Procesamiento de imágenes de Color del Mar 

Integrantes: Arena Maximiliano, Delgado Ana Laura, Urquiza Josefina (Grupo 8)


## 1. Corrección Atmosférica (CA) de imágenes MODIS-Aqua

El proceso de corrección atmosférica elimina los componentes (2) y (3), que son considerados como ruido en este contexto y son generados por la dispersión producida tanto por las moléculas del aire como por las partículas (aerosoles). La radiación que emerge de la capa superficial del mar, la componente (4) en la figura, es la única que contiene información sobre las sustancias presentes en el agua. Las características espectrales de la radiancia que emerge del agua (que llamaremos radiancia marina de aquí en más) dependen de la absorción y dispersión de la luz visible de los distintos componentes que se encuentren en el
agua (e.g. materia orgánica disuelta y material particulado en suspensión, incluyendo las células vivas del fitoplancton, sedimento inorgánico y detrito particulado).
![image](./images/imagen1.png)

## Descarga de imagenes L1A
Estas imágenes se distribuyen en los niveles L1A y L2.
Los datos auxiliares necesarios y que se utilizan en la corrección atmosférica comprenden la concentración de ozono presente en la atmósfera y los datos meteorológicos, tales como la dirección meridional y zonal del viento, su velocidad, la presión atmosférica y la humedad relativa.
![image](./images/imagen4.png)

https://oceancolor.gsfc.nasa.gov/

## Software de CA
Para el procesamiento de las imágenes MODIS existe un programa de distribución libre llamado SeaDAS 
https://seadas.gsfc.nasa.gov/downloads/

![image](./images/imagen2.jpg)

## Cadena de procesamiento
Utilizando la rutina del programa SeaDAS junto con los datos auxiliares se procesa la imagen nivel 1A a 1B junto con el archivo GEO (para las MODIS) con el fin de obtener la imagen nivel 2 con el producto geofísico deseado (por ejemplo concentración de Cla).
En este procesamiento se enmascara tanto la tierra como las nubes ya que los pixeles así identificados no poseen información útil para el estudio del mar. También se obtienen una serie de indicadores de calidad o flags para cada píxel

![image](./images/imagen3.png)

## Batch Process Seadas

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
import os
from netCDF4 import Dataset
import ast
import glob

import warnings
warnings.filterwarnings("ignore")
print("librerias ok")

librerias ok


In [8]:
#Armo lista de archivos 
imgList = glob.glob('./data/raw_data/L1/*L1A_LAC.x.hdf')

pathSeaDAS   = '/home/maxi/seadas8/ocssw' 

# imgList

In [10]:
#Seteo el tipo de corrección Atmosferica: NIRGW94 / NIR-IT / NIR-SWIR / SWIRGW94

l2genAC = {\
 'NIR_GW94'      :                          \
{'l2prod'        :'chlor_a',\
 'aer_opt'       :-1                       ,\
 'aer_wave_short':1240                     ,\
 'aer_wave_long' :2130                    }}

boxRoi = {'N': -38.94,'S': -39.23,'W': -61.98,'E': -61.50}

l2genCommon = {\
'brdf_opt'      :0          ,\
'maskhilt'      :0          ,\
'maskglint'     :0          ,\
'cloud_wave'    :2130       ,\
'cloud_thresh'  :0.018      ,\
'resolution'    :1000       ,\
'north'         :boxRoi['N'],\
'south'         :boxRoi['S'],\
'west'          :boxRoi['W'],\
'east'          :boxRoi['E']}


In [11]:
imgProcessList = ['Pre-process','AC SeaDAS']
coverage = 'LAC'

In [12]:
#% Preprocesarla con SeaDAS
for img in imgList:
        if 'Pre-process' in imgProcessList:
            preProcess = {\
            'GEO':['modis_GEO', ' -o ' + img + '.GEO'],\
            'ANC':['getanc'   , ''                                     ],\
            'L1B':['modis_L1B', \
                   ' '    + img + '.GEO'  \
                 + ' -o ' + img + '.L1B_' + coverage ]}
            for pr in preProcess.keys():
                print(img)
                procStr = pathSeaDAS + '/ocssw_src/scripts/' + preProcess[pr][0] + '.py ' + img + preProcess[pr][1]
                os.system(procStr)
                
#% Corregirla atmosfericamente con SeaDAS
        if 'AC SeaDAS' in imgProcessList:
            for ac in ['NIR_GW94']:
                print('IMG: ' + img + '; AC: ' + ac)
    
                l2genStrs = {}
                l2genStr = pathSeaDAS  + '/bin/l2gen'                      + \
                  ' ifile='   + img + '.L1B_' + coverage + \
                  ' geofile=' + img + '.GEO'             + \
                  ' ofile='   + img + ac + 'chl'
                   
                for key in l2genAC[ac].keys():
                    val = l2genAC[ac][key]
                    l2genStr = l2genStr + ' ' + key + '=' + str(val)
                
                for key in l2genCommon.keys():
                    val = l2genCommon[key]
                    l2genStr = l2genStr + ' ' + key + '=' + str(val)
                
                l2genStrs[ac] = l2genStr
                os.system(l2genStrs[ac])

./data/raw_data/L1/A2021193183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021193183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021193183000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021193183000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021052172500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021052172500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021052172500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021052172500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021209183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021209183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021209183000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021209183000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021208175000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021208175000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021208175000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021208175000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021173172000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021173172000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021173172000.L1A_LAC.x.hdf
IMG: ./data/

./data/raw_data/L1/A2021100172500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021100172500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021039175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021039175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021039175500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021039175500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021233174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021233174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021233174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021233174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021121174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021121174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021121174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021121174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021025174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021025174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021025174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021025174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021200184000.L1A_LAC

./data/raw_data/L1/A2021205185500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021205185500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021205185500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021027173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021027173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021027173000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021027173000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021163182000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021163182000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021163182000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021163182000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021091173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021091173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021091173000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021091173000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021197181000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021197181000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021197181000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021197181000.L1A_LAC.x.hdf; AC: NI

./data/raw_data/L1/A2021294185000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021294185000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021001183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021001183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021001183000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021001183000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021005181000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021005181000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021005181000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021005181000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021269185500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021269185500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021269185500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021269185500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021093172000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021093172000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021093172000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021093172000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021336175000.L1A_LAC

./data/raw_data/L1/A2021204181500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021204181500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021137174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021137174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021137174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021137174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021220181500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021220181500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021220181500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021220181500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021105174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021105174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021105174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021105174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021243182000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021243182000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021243182000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021243182000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021189172000.L1A_LAC

./data/raw_data/L1/A2021340190500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021340190500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021297174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021297174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021297174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021297174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021311175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021311175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021311175500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021311175500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021284181500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021284181500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021284181500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021284181500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021324190500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021324190500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021324190500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021324190500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021133181000.L1A_LAC

./data/raw_data/L1/A2021074182500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021074182500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021085181000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021085181000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021085181000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021085181000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021304175000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021304175000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021304175000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021304175000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021075173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021075173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021075173000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021075173000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021123173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021123173000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021123173000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021123173000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021273183000.L1A_LAC

./data/raw_data/L1/A2021285172000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021285172000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021285172000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021329174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021329174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021329174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021329174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021078180000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021078180000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021078180000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021078180000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021307182000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021307182000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021307182000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021307182000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021186182500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021186182500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021186182500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021186182500.L1A_LAC.x.hdf; AC: NI

./data/raw_data/L1/A2021176175000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021176175000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021176175000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021089174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021089174500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021089174500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021089174500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021098173500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021098173500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021098173500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021098173500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021040184000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021040184000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021040184000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021040184000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021177183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021177183000.L1A_LAC.x.hdf
./data/raw_data/L1/A2021177183000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021177183000.L1A_LAC.x.hdf; AC: NI

./data/raw_data/L1/A2021203173000.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021203173000.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021210173500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021210173500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021210173500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021210173500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021076181500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021076181500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021076181500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021076181500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021103175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021103175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021103175500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021103175500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021295175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021295175500.L1A_LAC.x.hdf
./data/raw_data/L1/A2021295175500.L1A_LAC.x.hdf
IMG: ./data/raw_data/L1/A2021295175500.L1A_LAC.x.hdf; AC: NIR_GW94
./data/raw_data/L1/A2021291182000.L1A_LAC

## L2 A L3

In [29]:
#Armo lista de archivos 
# path = '/home/ana/Curso_PDI_linux/MODIS_py'
imgList = glob.glob('/home/maxi/Curso_PDI_linux/MODIS_py/*L2_LAC')
pathSeaDAS   = '/home/maxi/seadas8/ocssw'
imgProcessList = ['L2bin','L3gen']
coverage = 'LAC'

In [14]:
#Seteo el producto; la resolucion y los flags que quiero aplicar a las imagenes

l2bin = {\
 'BIN'      :                          \
{'l3bprod'        :'chlor_a',\
 'resolve'       :1                       ,\
 'flaguse': 'ATMFAIL,LAND,HIGLINT,STRAYLIGHT'                  }}


#seteo map: area, proyeccion; resolucion, metodo de interpolacion   
l3mapgen = {\
 'MAP'      :                          \
{'prod'        :'chlor_a'      ,\
 'deflate':  '4' ,\
 'scale_type':  'linear'      ,\
 'projection':  'platecarree' ,\
 'resolution':  '1' + 'km'      ,\
 'interp':  'nearest'         ,\
 'west': -63                   ,\
 'east': -57   ,\
 'north': -37  ,\
 'south': -42            }}     

In [31]:
for img in imgList:    
     if 'L2bin' in imgProcessList:
            for bi in ['BIN']:
                print(img)
            
                l2binStrs = {}
                l2binStr = pathSeaDAS  + '/bin/l2bin'                      + \
                  ' ifile='   + img + '.L2_LAC' + \
                  ' ofile='   + img.replace('L1A_LAC.x.hdf','L2_bin')           
             
                for key in l2bin[bi].keys():
                    val = l2bin[bi][key]
                    l2binStr = l2binStr + ' ' + key + '=' + str(val)
                
                l2binStrs[bi] = l2binStr
                os.system(l2binStrs[bi])  
       
     if 'L3gen' in imgProcessList:
            for ge in ['MAP']:
                print(img)
               
                l3mapgenStrs = {}
                l3mapgenStr = pathSeaDAS  + '/bin/l3mapgen'                      + \
                 'ifile=' + img + '.L2_LAC' + \
                 'ofile=' + img.replace('L1A_LAC.x.hdf','L3') 
               
                for key in l3gen[ge].keys():
                    val = l3gen[ge][key]
                    l3genStr = l3genStr + ' ' + key + '=' + str(val)
                
                l3genStrs[ge] = l3genStr
                os.system(l3genStrs[ge])  

    